<a href="https://colab.research.google.com/github/Jeffstr00/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-04-05 19:36:25--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2022-04-05 19:36:25 (6.49 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
# Import SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# Read in the review set from S3 as a DataFrame
from pyspark import SparkFiles
url = "https://jeffstr00-bucket.s3.amazonaws.com/amazon_reviews_us_Books_v1_00.tsv"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   25933450| RJOVP071AVAJO|0439873800|      84656342|There Was an Old ...|           Books|          5|            0|          0|   N|                Y|          Five Stars|I love it and so ...| 2015-08-31|
|         US|    1801372|R1ORGBETCDW3AI|1623953553|     729938122|      I Saw a Friend|           Books|          5|    

In [5]:
# Change Star Rating column from string to integer
from pyspark.sql.types import IntegerType
df = df.withColumn("star_ratingTmp", df["star_rating"].cast(IntegerType())).drop("star_rating").withColumnRenamed("star_ratingTmp", "star_rating")
# Rearrange columns so that Star Rating is back in place
df = df.select("marketplace", "customer_id", "review_id", "product_id", "product_parent", "product_title", "product_category", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase", "review_headline", "review_body", "review_date")
df.schema

StructType(List(StructField(marketplace,StringType,true),StructField(customer_id,IntegerType,true),StructField(review_id,StringType,true),StructField(product_id,StringType,true),StructField(product_parent,IntegerType,true),StructField(product_title,StringType,true),StructField(product_category,StringType,true),StructField(star_rating,IntegerType,true),StructField(helpful_votes,IntegerType,true),StructField(total_votes,IntegerType,true),StructField(vine,StringType,true),StructField(verified_purchase,StringType,true),StructField(review_headline,StringType,true),StructField(review_body,StringType,true),StructField(review_date,StringType,true)))

In [6]:
# Create the vine_table. DataFrame
# vine_df = df.select([])
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RJOVP071AVAJO|          5|            0|          0|   N|                Y|
|R1ORGBETCDW3AI|          5|            0|          0|   N|                Y|
| R7TNRFQAOUTX5|          5|            0|          0|   N|                Y|
|R2GANXKDIFZ6OI|          5|            0|          0|   N|                N|
|R2NYB6C3R8LVN6|          5|            2|          2|   N|                Y|
|R13U5PBJI1H94K|          2|            1|          1|   N|                N|
|R1H8UVH990F8VE|          5|            2|          2|   N|                N|
|R2MC0N30WZMRQ5|          5|            0|          0|   N|                Y|
| R2NO2HXK16Y4J|          5|            0|          0|   N|                Y|
|R245YIAVJK82ZL|          5|            0|          0|   N|     

In [7]:
# count number of rows in Vine DF to ensure that everything loaded properly
vine_df.count()

10319090

In [8]:
# filter data to retrieve all rows where total_votes is greater or equal to 20
substantial_df = vine_df.filter("total_votes>=20")
substantial_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3T6LR4H11XRAN|          5|           26|         26|   N|                Y|
|R3QFNFMFWOQIBO|          3|           15|         25|   N|                Y|
|R24S2SV0ZIFIMV|          1|           41|         78|   N|                N|
|R2QAPWF1M5863L|          1|            5|         23|   N|                N|
|R3OCONKYE1A047|          5|           33|         38|   N|                N|
|R28WZIA84DUP6C|          2|            4|         43|   N|                N|
|R2U7YNJZ5JHWVD|          5|           23|         25|   N|                Y|
|R1XVYF74WCJKO3|          1|           47|         49|   N|                N|
|R3KKWNG5VABDYC|          5|           31|         32|   Y|                N|
|R3KK9MHFLY2BVW|          5|           21|         22|   N|     

In [11]:
# filter DataFrame to retrieve all rows where the number of helpful votes divided by total_votes is greater than or equal to 50%
helpful_df = substantial_df.filter("(helpful_votes/total_votes)>=.5")
helpful_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3T6LR4H11XRAN|          5|           26|         26|   N|                Y|
|R3QFNFMFWOQIBO|          3|           15|         25|   N|                Y|
|R24S2SV0ZIFIMV|          1|           41|         78|   N|                N|
|R3OCONKYE1A047|          5|           33|         38|   N|                N|
|R2U7YNJZ5JHWVD|          5|           23|         25|   N|                Y|
|R1XVYF74WCJKO3|          1|           47|         49|   N|                N|
|R3KKWNG5VABDYC|          5|           31|         32|   Y|                N|
|R3KK9MHFLY2BVW|          5|           21|         22|   N|                N|
|R2OEP8MGX5K370|          5|           30|         34|   N|                Y|
| RTX63ENBKWCXF|          1|           28|         35|   N|     

In [12]:
# filter DataFrame to retrieve all rows where a review was written as part of the Vine program (paid)
paid_df = helpful_df.filter("vine == 'Y'")
paid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3KKWNG5VABDYC|          5|           31|         32|   Y|                N|
|R397PIJYYVJ7PD|          4|           31|         32|   Y|                N|
| RP6Y2CTIFUYD0|          2|           31|         32|   Y|                N|
|R1FYRNM5U0S2VZ|          3|           20|         24|   Y|                N|
| R1ME19UY4UNAO|          5|           53|         57|   Y|                N|
|R1JW7ZHT2VLT1E|          3|           40|         45|   Y|                N|
| R98UYXY43TURV|          4|           46|         50|   Y|                N|
|R34VXM3EJDS78Z|          3|          374|        410|   Y|                N|
|R1QX8UOW5JO6L0|          4|           20|         21|   Y|                N|
| R5Y5SXCRQ2GNH|          3|           34|         37|   Y|     

In [13]:
# filter DataFrame to retrieve all rows where a review was NOT written as part of the Vine program (unpaid)
unpaid_df = helpful_df.filter("vine == 'N'")
unpaid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3T6LR4H11XRAN|          5|           26|         26|   N|                Y|
|R3QFNFMFWOQIBO|          3|           15|         25|   N|                Y|
|R24S2SV0ZIFIMV|          1|           41|         78|   N|                N|
|R3OCONKYE1A047|          5|           33|         38|   N|                N|
|R2U7YNJZ5JHWVD|          5|           23|         25|   N|                Y|
|R1XVYF74WCJKO3|          1|           47|         49|   N|                N|
|R3KK9MHFLY2BVW|          5|           21|         22|   N|                N|
|R2OEP8MGX5K370|          5|           30|         34|   N|                Y|
| RTX63ENBKWCXF|          1|           28|         35|   N|                N|
|R38I0D0J8O6LB6|          5|          101|        102|   N|     

In [25]:
# determine total number of reviews, 5-star reviews, and percentage of 5-star reviews for paid reviews
paid_total = paid_df.count()
paid_five = paid_df.filter("star_rating=5").count()
paid_five_percent = (paid_five/paid_total)*100

In [31]:
# display paid results
print("Paid (Vine) Review Summary")
print("======================================================")
print(f"Total number of paid (Vine) reviews: \033[1m{paid_total}\033[0m")
print(f"Number of paid 5-star reviews: \033[1m{paid_five}\033[0m")
print(f"Percentage of paid reviews with 5-star reviews: \033[1m{paid_five_percent:.2f}%\033[0m")

Paid (Vine) Review Summary
Total number of paid (Vine) reviews: 5012
Number of paid 5-star reviews: 2031
Percentage of paid reviews with 5-star reviews: 40.52%


In [27]:
# determine total number of reviews, 5-star reviews, and percentage of 5-star reviews for unpaid reviews
unpaid_total = unpaid_df.count()
unpaid_five = unpaid_df.filter("star_rating=5").count()
unpaid_five_percent = (unpaid_five/unpaid_total)*100

In [32]:
# display paid results
print("Unpaid (Non-Vine) Review Summary")
print("======================================================")
print(f"Total number of unpaid (non-Vine) reviews: \033[1m{unpaid_total}\033[0m")
print(f"Number of unpaid 5-star reviews: \033[1m{unpaid_five}\033[0m")
print(f"Percentage of unpaid reviews with 5-star reviews: \033[1m{unpaid_five_percent:.2f}%\033[0m")

Unpaid (Non-Vine) Review Summary
Total number of unpaid (non-Vine) reviews: 109297
Number of unpaid 5-star reviews: 49967
Percentage of unpaid reviews with 5-star reviews: 45.72%


# Additional Analysis

In [36]:
# determine how many one star reviews are paid (and the percent)
paid_one = paid_df.filter("star_rating=1").count()
paid_one_percent = (paid_one/paid_total)*100

In [37]:
# display paid, 1-Star results
print(f"Total number of paid (Vine) reviews: \033[1m{paid_total}\033[0m")
print(f"Number of paid 1-star reviews: \033[1m{paid_one}\033[0m")
print(f"Percentage of paid reviews with 1-star reviews: \033[1m{paid_one_percent:.2f}%\033[0m")

Total number of paid (Vine) reviews: 5012
Number of paid 1-star reviews: 217
Percentage of paid reviews with 1-star reviews: 4.33%


In [38]:
# determine how many one star reviews are unpaid (and the percent)
unpaid_one = unpaid_df.filter("star_rating=1").count()
unpaid_one_percent = (unpaid_one/unpaid_total)*100

In [39]:
# display unpaid, 1-Star results
print(f"Total number of unpaid (non-Vine) reviews: \033[1m{unpaid_total}\033[0m")
print(f"Number of unpaid 1-star reviews: \033[1m{unpaid_one}\033[0m")
print(f"Percentage of unpaid reviews with 1-star reviews: \033[1m{unpaid_one_percent:.2f}%\033[0m")

Total number of unpaid (non-Vine) reviews: 109297
Number of unpaid 1-star reviews: 28845
Percentage of unpaid reviews with 1-star reviews: 26.39%
